In [64]:
import numpy as np
import pandas as pd
from datetime import datetime
import calendar
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [65]:
train = train[train['weather'] != 4]

In [66]:
all_data = pd.concat([train, test], ignore_index=True)
# feature_names=list(test)
# df_train=train[feature_names]
# all_data=pd.concat((df_train, test))

In [67]:
# all_data.drop(['season', 'workingday'], axis=1, inplace=True)


In [68]:
all_data['tempDate'] = all_data.datetime.apply(lambda x:x.split())

all_data['year'] = all_data['tempDate'].apply(lambda x: x[0].split('-')[0])
all_data['month'] = all_data['tempDate'].apply(lambda x: x[0].split('-')[1])
all_data['hour'] = all_data['tempDate'].apply(lambda x: x[1].split(':')[0])
# all_data['weekday'] = all_data.tempDate.apply(lambda x:calendar.day_name[datetime.strptime(x[0], "%Y-%m-%d").weekday()])

all_data = all_data.drop('tempDate', axis=1)
# all_data = all_data.drop('datetime', axis=1)

In [69]:
# atemp
def predict_atemp(data):
    validData = data[(data["atemp"] > 15) | (data["temp"] <= 20)]
    invalidData = data[(data["atemp"] <= 15) & (data["temp"] > 20)]

    lin_atemp = LinearRegression()
    lin_atemp.fit(validData[["temp"]], validData[["atemp"]])

    predict_atemp = lin_atemp.predict(invalidData[["temp"]])
    
    invalidData.loc[:, "atemp"] = predict_atemp

    data = pd.concat([validData, invalidData], ignore_index=True)

    return data

In [70]:
all_data = predict_atemp(all_data)

In [71]:
# windspeed
from sklearn.ensemble import RandomForestClassifier

data = all_data
trainWind0 = data.loc[data["windspeed"] == 0]
trainWindNot0 = data.loc[data["windspeed"]!=0]

def predict_windspeed(data):
    dataWind0 = data.loc[data["windspeed"] == 0]
    dataWindNot0 = data.loc[data["windspeed"] != 0]

    windColumns = ["weather", "humidity", "month", "temp", "atemp"]
    # dataWindNot0.loc[:, "windspeed"] = dataWindNot0["windspeed"].astype("str")

    rf_wind = RandomForestRegressor()
    rf_wind.fit(dataWindNot0[windColumns], dataWindNot0["windspeed"])
    wind0 = rf_wind.predict(X=dataWind0[windColumns])
    
    # predictWind0 = dataWind0
    # predictWindNot0 = dataWindNot0

    dataWind0.loc[:, "windspeed"] = wind0
    data = pd.concat([dataWindNot0, dataWind0])
    # data["windspeed"] = data["windspeed"].astype("float")
    data.reset_index(inplace = True)
    data.drop("index", inplace = True, axis = 1)
    
    return data

In [72]:
all_data = predict_windspeed(all_data)
all_data.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,year,month,hour
0,2011-01-01 05:00:00,1,0,0,2,9.84,12.880,75,6.0032,0.0,1.0,1.0,2011,01,05
1,2011-01-01 10:00:00,1,0,0,1,15.58,19.695,76,16.9979,12.0,24.0,36.0,2011,01,10
2,2011-01-01 11:00:00,1,0,0,1,14.76,16.665,81,19.0012,26.0,30.0,56.0,2011,01,11
3,2011-01-01 12:00:00,1,0,0,1,17.22,21.210,77,19.0012,29.0,55.0,84.0,2011,01,12
4,2011-01-01 13:00:00,1,0,0,2,18.86,22.725,72,19.9995,47.0,47.0,94.0,2011,01,13


In [73]:
# humidity
def predict_humidity(data):
    dataHum0 = data[data["humidity"]==0]
    dataHumNot0 = data[data["humidity"]!=0]

    humidityColumns = ["weather", "windspeed", "month", "temp", "atemp"]

    rf_humidity = RandomForestRegressor()
    rf_humidity.fit(dataHumNot0[humidityColumns], dataHumNot0["humidity"])

    predictHumidity0 = rf_humidity.predict(X= dataHum0[humidityColumns])
    dataHum0.loc[:, "humidity"] = predictHumidity0

    data = pd.concat([dataHumNot0, dataHum0], ignore_index=True)
    # data.reset_index(inplace = True)
    # data.drop("index", inplace = True, axis = 1)

    return data


In [74]:
all_data = predict_humidity(all_data)

In [75]:
oheFeatures = ["holiday", "weather", "season", "workingday"]
all_data = pd.get_dummies(all_data, columns=oheFeatures)

In [76]:
# new_train = all_data[~pd.isnull(all_data['count'])]
# new_test = all_data[pd.isnull(all_data['count'])]
# new_train = all_data[:train.shape[0]]
# new_test = all_data[train.shape[0]:]

new_train = all_data[pd.notnull(all_data['count'])].sort_values(by=["datetime"])
new_test = all_data[~pd.notnull(all_data['count'])].sort_values(by=["datetime"])

In [77]:
reg_target = new_train["registered"]
cas_target = new_train["casual"]


In [78]:
dropFeatures = ["casual", "count", "datetime", "registered"]
new_train  = new_train.drop(dropFeatures, axis=1)
new_test  = new_test.drop(dropFeatures, axis=1)

In [79]:
reg_target = np.log(reg_target+1)
cas_target = np.log(cas_target+1)

In [80]:
# import pandas as pd
# import seaborn as sns
# import matplotlib as mpl
# import matplotlib.pyplot as plt


# mpl.rc('font', size = 15)
# figure, axes = plt.subplots(nrows = 2, ncols = 2)
# plt.tight_layout()
# figure.set_size_inches(7,6)

# sns.regplot(x = 'temp', y = 'atemp', data = train, ax = axes[0,0], scatter_kws= {'alpha': 0.5}, line_kws={'color': 'blue'})
# sns.regplot(x = 'windspeed', y = 'count', data = train, ax = axes[1,0], scatter_kws= {'alpha': 0.3}, line_kws={'color': 'blue'})
# sns.regplot(x = 'humidity', y = 'count', data = train, ax = axes[1,1], scatter_kws= {'alpha': 0.3}, line_kws={'color': 'blue'})

In [81]:
import numpy as np
def rmsle_score(preds, true):
    rmsle_score = (np.sum((np.log1p(preds)-np.log1p(true))**2)/len(true))**0.5
    return rmsle_score

In [82]:
from sklearn.metrics import make_scorer
RMSLE = make_scorer(rmsle_score)

In [83]:
from sklearn.model_selection import train_test_split 

reg_X_train, reg_X_validation, reg_y_train, reg_y_validation = train_test_split(new_train, reg_target, test_size=0.2, shuffle=True)
cas_X_train, cas_X_validation, cas_y_train, cas_y_validation = train_test_split(new_train, cas_target, test_size=0.2, shuffle=True)

In [84]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestRegressor

param_Rf =  {'min_samples_split' : [3,4,6,10], 'n_estimators' : [70,100], 'random_state': [5] }
cas_RF = GridSearchCV(RandomForestRegressor(random_state=1), param_Rf, cv=10, scoring = RMSLE)
cas_RF.fit(cas_X_train, cas_y_train)

GridSearchCV(cv=10, estimator=RandomForestRegressor(random_state=1),
             param_grid={'min_samples_split': [3, 4, 6, 10],
                         'n_estimators': [70, 100], 'random_state': [5]},
             scoring=make_scorer(rmsle_score))

In [85]:
reg_RF = GridSearchCV(RandomForestRegressor(random_state=1), param_Rf, cv=10, scoring = RMSLE)
reg_RF.fit(reg_X_train, reg_y_train)

GridSearchCV(cv=10, estimator=RandomForestRegressor(random_state=1),
             param_grid={'min_samples_split': [3, 4, 6, 10],
                         'n_estimators': [70, 100], 'random_state': [5]},
             scoring=make_scorer(rmsle_score))

In [86]:
reg_RF_yhat = reg_RF.predict(reg_X_validation)
reg_s_RF = rmsle_score(reg_RF_yhat, reg_y_validation)
reg_s_RF

0.09879192835447517

In [87]:
cas_RF_yhat = cas_RF.predict(cas_X_validation)
cas_s_RF = rmsle_score(cas_RF_yhat, cas_y_validation)
cas_s_RF

0.22377516568814004

In [88]:
reg_pred_rf = reg_RF.predict(new_test)
cas_pred_rf = cas_RF.predict(new_test)

In [89]:
sub = pd.DataFrame()
sub['datetime'] = test['datetime']
sub['count'] = [max(0, x) for x in np.exp(reg_pred_rf) + np.exp(cas_pred_rf)]


sub.to_csv('submission1.csv', index=False)

In [90]:
# RF.predict(new_test)

In [91]:
param = {'alpha':[1e-03,0.01,0.1,0.5,0.8,1], 'tol':[1e-06,1e-05,5e-05,1e-04,5e-04,1e-03]}

In [92]:
from sklearn.linear_model import Lasso

lasso = make_pipeline(GridSearchCV(Lasso(random_state=1),param,
                                  cv=10, scoring = RMSLE))

In [93]:
lasso.fit(X_train,y_train)

In [ ]:
la_yhat = lasso.predict(X_validation)

In [ ]:
s_lasso = rmsle_score(la_yhat,y_validation)
s_lasso

0.28036850925788315